# Szövegbányászat (Natural Language Processing, NLP)
A szociális médiában (is) óriási mennyiségű adat érhető el strukúrálatlan, például szöveges formában. Ezen adatok automatikus feldolgozását nevezzük szövegbányászatnak vagy nyelvtechnológiának (angolul Natural Language Processing).

Az emberi nyelvek gyönyörűek. Ugyanazt száz féle képpen ki tudjuk fejezni és egy szó vagy kifejezés sok mindent tud jelenteni más mondatkörnyezetben. Az emberek által, emberi olvasásra írt szövegek gépi *értelmezése* máig megoldatlan probléma. Azonban a szövegek feldolgozásában rengeteget fejlődött a technológia az elmúlt 20 évben.


### További anyagok:
*   D. Jurafsky: Intorduction to NLP [youtube](https://www.youtube.com/watch?v=oWsMIW-5xUc)
*   HuSpaCy magyar nyelvű szövegfeldolgozó rendszer [demo oldala](https://huggingface.co/spaces/huspacy/demo)




# Tokenizálás
Dokumentumok hosszú string reprezentációjával nem lehet adatelemzésben dolgozni, szükség van azok előfeldolgozására, olyan egységekre bontásra, ami már kezelhető: mondatok és szavak lesznek az egység.

Az [NLTK](https://www.nltk.org/) és a [spaCy](https://spacy.io/) a leggyakrabban használt python csomagok szövegfeldolgozásban. Kettejük egy összehasonlítása [itt](https://www.activestate.com/blog/natural-language-processing-nltk-vs-spacy/)



### Adatok betöltése – Kaggle-SMS-Spam-Collection-Dataset

A Kaggle SMS Spam Collection Dataset egy népszerű nyilvános adathalmaz, amelyet SMS-ek spam detekciójára használnak.
- Tisztított, valódi SMS-eket tartalmaz, amelyeket két kategóriába soroltak:
  - "ham" (normál üzenet)
  - "spam" (kéretlen reklám, csalás, stb.)

 Eredeti forrás: A dataset a UCI Machine Learning Repository-ból származik, amelyet kutatási célokra gyűjtöttek össze.

Felhasználási területek:
- Spam detekció gépi tanulással
- Szövegbányászat (NLP)
- Véleménydetekció és osztályozás



In [ ]:
import pandas as pd

# SMS Spam Dataset betöltése
url = "https://github.com/mohitgupta-1O1/Kaggle-SMS-Spam-Collection-Dataset-/raw/refs/heads/master/spam.csv"

# CSV fájl betöltése pandas DataFrame-be
# A 'latin1' kódolást használjuk, és figyelmen kívül hagyjuk a hibás sorokat
df = pd.read_csv(url, sep=",", header=0, encoding='latin1', on_bad_lines='skip')

# Csak a két szükséges oszlopot tartjuk meg ('label' és 'message')
df = df.iloc[:, :2].rename(columns={'v1': 'label', 'v2': 'message'})

# Az első dokumentum szövege
print("Első dokumentum szövege:\n")
print(df.message[0])



### Tokenizálás – Szavakra és mondatokra bontás
- A tokenizálás az első lépés az NLP-ben, ahol a szöveget kisebb egységekre (tokenekre) bontjuk.
- Használhatunk nltk, spaCy, vagy scikit-learn megoldásokat.

### Tokenizálás NLTK segítségével
- A szöveget mondatokra (sent_tokenize) és szavakra (word_tokenize) bontjuk.

In [ ]:
#pip install --upgrade nltk

In [ ]:
import nltk
import os
nltk.data.path.append(os.path.expanduser('~/nltk_data'))
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize

# Első dokumentum kiválasztása
text = df.message[0]

In [ ]:
# Mondatokra bontás
sentences = sent_tokenize(text)
print("\nMondatok száma:", len(sentences))
display("Első mondat:", sentences[0])

In [ ]:
# Szavakra bontás
words = word_tokenize(text)
print("\nTokenek száma:", len(words))
print("Első 10 szó:", words[:10])

Mit csinálunk?

- Mondatokra bontás (sent_tokenize) -> Megszámoljuk, hány mondat van.
- Szavakra bontás (word_tokenize) -> Megnézzük az első 10 szót.

### Tokenizálás SpaCy segítségével

A SpaCy egy erősebb NLP könyvtár, amely gyorsabb és több funkcióval rendelkezik.

In [ ]:
import spacy

# Angol nyelvű NLP modell betöltése
nlp = spacy.load("en_core_web_sm")

# SpaCy tokenizálás
doc = nlp(text)

# Mondatok kiíratása
sentences_spacy = list(doc.sents)
print("\nSpaCy mondatok száma:", len(sentences_spacy))
print("Első mondat:", sentences_spacy[0])

# Szavak kiíratása
tokens_spacy = [token.text for token in doc]
print("\nSpaCy első 10 szó:", tokens_spacy[:10])


Miért jó a SpaCy?

- Gyorsabb, mint NLTK, mert optimalizált C-kódban fut.
- Nemcsak szavakat, hanem entitásokat, szófajokat is felismer.

### Tokenizálás Scikit-learn segítségével

A CountVectorizer automatikusan tokenizál és előkészíti az adatokat későbbi modellekhez.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df.message)

print("\nTokenek száma az egész dataset-ben:", len(vectorizer.get_feature_names_out()))
print("Első 10 token:", vectorizer.get_feature_names_out()[:10])


Mit csinálunk?

- A CountVectorizer egy BoW (Bag-of-Words) modellt készít.
- Minden egyes egyedi szót megtalálja és megszámolja az előfordulását.

Mit jelent a fura token lista?

- A vektorizáló valójában az SMS-ek tartalmát próbálta átalakítani tokenekké, de mivel az üzenetekben sok szám is szerepel, a következő történhetett:
['00' '000' '000pes' '008704050406' '0089' '0121' '01223585236']

Megoldás – Számok kizárása
- Ha nem szeretnéd, hogy számokat is tokenizáljon, akkor a CountVectorizer() beállításaival ezt szabályozhatod:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b[a-zA-Z]+\b')
X = vectorizer.fit_transform(df.message)

print("\nTokenek száma az egész dataset-ben:", len(vectorizer.get_feature_names_out()))
print("Első 10 token:", vectorizer.get_feature_names_out()[:10])


### Tokenizálás vizualizálása – Szavak gyakorisági eloszlása

Most készítsünk egy hisztogramot a leggyakoribb szavakról.

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Tokenek gyakorisági elemzése
word_counts = Counter(words)
common_words = word_counts.most_common(15)

# Adatok elkülönítése a vizualizációhoz
words, counts = zip(*common_words)

plt.figure(figsize=(10, 5))
sns.barplot(x=list(words), y=list(counts), palette="coolwarm")
plt.xticks(rotation=45)
plt.xlabel("Szavak")
plt.ylabel("Előfordulás")
plt.title("Leggyakoribb 15 szó")
plt.show()


Mit látunk?

- A legtöbbször előforduló szavak gyakoriságát a datasetben.
- A későbbiekben stopwords szűrést kell alkalmazni, hogy a "the", "and" stb. ne legyen benne.


A tokenizálás az NLP első lépése, amely segít a szöveget strukturált adatokká alakítani.
A következő lépés: Stopwords eltávolítása és Szófaji elemzés (POS tagging).


### Miért nem egyértelmű a tokenizálás? Miért léteznek különböző algoritmusok?

- Azért mert különböző nyelveken másképp lehetnek a szóhatárok (magyar "-e"), például rövidítést jelentő pont a token része ("U.S.A." vagy "kft."), míg mondatvégi a írásjel nem.
- A szöveg típusa is megkövetelhet különböző tokenizálókat, pl. szociális médiában az emotikonok és URLeket egyben kell tartni de a camelcase szavakat (pl. JoMunkahozIdoKell) tokenizáljuk, kémiai szövegekben a kötőjel egy molekula nevében nem szóhatár, stb.

### Stopwords eltávolítása

- A stopwords azok a szavak, amelyek gyakran előfordulnak, de általában nem hordoznak fontos információt (pl. "the", "is", "in", "on", "and").
- Stopwords eltávolítása segít csökkenteni az adatdimenziót és jobb modelleket készíteni.

### Stopwords eltávolítása NLTK segítségével
- NLTK beépített angol stopwords listáját használjuk.

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Stopwords lista betöltése
stop_words = set(stopwords.words("english"))

# Tokenizálás (szavakra bontás)
words = word_tokenize(text)

# Stopwords eltávolítása
filtered_words = [word for word in words if word.lower() not in stop_words]

print("\nEredeti szólista:", words[:20])
print("\nStopwords eltávolítása után:", filtered_words[:20])


Mit csinálunk?

- Tokenizáljuk a szöveget (word_tokenize()).
- Eltávolítjuk a gyakori angol stopwords szavakat (pl. "the", "is", "on", "and").
- Az eredmény egy tisztább szöveg, amely kevesebb felesleges szót tartalmaz.

### Stopwords eltávolítása SpaCy segítségével
- SpaCy beépített stopwords listáját használjuk, amely gyorsabb és pontosabb lehet.

In [ ]:
import spacy

# Angol nyelvű NLP modell betöltése
nlp = spacy.load("en_core_web_sm")

# SpaCy tokenizálás és stopwords eltávolítása
doc = nlp(text)
filtered_words_spacy = [token.text for token in doc if not token.is_stop]

print("\nStopwords eltávolítása után SpaCy-val:", filtered_words_spacy[:20])


Mit csinálunk?

- Tokenizáljuk a szöveget SpaCy-vel.
- Automatikusan eltávolítjuk a stopwords szavakat.
- Az eredmény egy tisztább szólista.

### Szófaji elemzés (POS Tagging)


- A szófaji elemzés (Part-Of-Speech Tagging, POS Tagging) meghatározza, hogy egy szó milyen szófaji kategóriába tartozik (pl. főnév, ige, melléknév).
- Ez segít megérteni a mondat szerkezetét és a fontosabb szavakat kiemelni.

### POS Tagging NLTK segítségével
- NLTK beépített POS címkézőjét használjuk.

In [ ]:
import nltk
import os

nltk.data.path.append(os.path.expanduser('~/nltk_data')) #vagy a megfelelő könyvtár
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
# Szófaji címkézés NLTK-val
pos_tags = nltk.pos_tag(filtered_words)
print("\nSzófaji címkék (NLTK):", pos_tags[:10])

Mit csinálunk?

- okenizáljuk a szöveget és eltávolítjuk a stopwords-t.
- POS címkézéssel megjelöljük a szavakat, hogy főnév (NN), ige (VB), melléknév (JJ) stb.

- bővebben a röviditésekről: https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

### POS Tagging SpaCy segítségével
- SpaCy beépített POS címkézője gyorsabb és pontosabb lehet.

In [ ]:
# SpaCy POS tagging
pos_tags_spacy = [(token.text, token.pos_) for token in doc]

print("\nSzófaji címkék (SpaCy):", pos_tags_spacy[:10])


Mit csinálunk?

- SpaCy automatikusan POS címkézi a szöveget.
- Az eredmény ember-barátabb és könnyebben értelmezhető.

### POS Tagging vizualizálása – Szófaji eloszlás
- Készítsünk egy oszlopdiagramot, amely megmutatja a szavak szófaji eloszlását!

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# POS címkék gyakorisági eloszlása
pos_counts = Counter([pos for word, pos in pos_tags_spacy])

# Adatok vizualizálása
plt.figure(figsize=(10, 5))
sns.barplot(x=list(pos_counts.keys()), y=list(pos_counts.values()), palette="coolwarm")
plt.xlabel("Szófaj")
plt.ylabel("Előfordulás")
plt.title("Szófaji címkék eloszlása")
plt.show()


Mit látunk?

- Mely szófajok dominálnak a szövegben.
- Ha például túl sok ige van, az egy akcióközpontú szöveget jelenthet.

### Stemming és a Lemmatizálás

- Még egy fontos előfeldolgozási lépés a szótövesítés (stemming, lemmatizáció)
- A cél, hogy a szavak különböző ragozott alakjait össze tudjuk vonni (pl. általában nem érdemes külön kezelni az 'asztalaitokra' és 'asztal' szóalakokat).
- A lemmatizáció az igazi nyelvtani értelemben vett szótő meghatározását jelenti. Ez nagyon bonyolult feladat tud lenni bizonyos nyelveken, pl. a magyarban ahol tőhangváltás is van (a 'madarak' szó szótöve a 'madár').
- Sokszor elég a szótőnek egy "közelítése", azaz egyszerű szabályokkal lecseréljük a szóalak végi karaktereket más karakterre.
- Ezt a közelítő (butább, de sokkal egyszerűbb és gyorsabb) hívjuk stemmelésnek (részletesen)

- Stemming (származtatás): A szó tövére vágása szabály alapú módszerekkel (pl. "running"-> "run").
- Lemmatizálás: A szó szótári alakjára történő visszavezetése (pl. "better" -> "good").
- A lemmatizálás pontosabb, mert a szó jelentését is figyelembe veszi.

### Stemming (Származtatás) NLTK segítségével
- NLTK két beépített stemmert tartalmaz: PorterStemmer és LancasterStemmer

In [ ]:
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize

# Tokenizálás
words = word_tokenize(text)

# Két különböző stemmer tesztelése
porter = PorterStemmer()
lancaster = LancasterStemmer()

# Példa stemming eredményekre
stemmed_porter = [porter.stem(word) for word in words[:20]]
stemmed_lancaster = [lancaster.stem(word) for word in words[:20]]

print("\nEredeti szavak:", words[:20])
print("\nPorter Stemming:", stemmed_porter)
print("\nLancaster Stemming:", stemmed_lancaster)


Mit csinálunk?

- Tokenizáljuk a szöveget (word_tokenize()).
- Kétféle stemming algoritmust tesztelünk:
  - Porter Stemmer: Precízebb, kevésbé agresszív.
  - Lancaster Stemmer: Agresszívebb, erősebben rövidít.
- Eredmény: A szavakat tövére csonkoljuk.

Mikor használjuk a stemminget?
- Amikor gyors és egyszerű megoldásra van szükség, például keresőmotorokhoz

### Lemmatizálás (Lemmatization) NLTK segítségével
- NLTK a WordNetLemmatizer függvényt használja, amely pontosabb, mint a stemming.

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

# Lemmatizer inicializálása
lemmatizer = WordNetLemmatizer()

# Példa lemmatizált szavakra
lemmatized_words = [lemmatizer.lemmatize(word) for word in words[:20]]

print("\nEredeti szavak:", words[:20])
print("\nLemmatizált szavak:", lemmatized_words)


Mit csinálunk?

- A WordNetLemmatizer visszaadja a szavak szótári alakját.
- Pl. "running" -> "run", "better" -> "better" (mert nincs szófaji megadva, ezt pontosíthatjuk).

Mikor jobb a lemmatizálás?

- Ha pontos eredményre van szükség (például szentiment elemzésnél).

### Lemmatizálás SpaCy segítségével
- A SpaCy lemmatizálója fejlettebb, és automatikusan felismeri a szófajokat is.

In [ ]:
import spacy

# SpaCy NLP modell betöltése
nlp = spacy.load("en_core_web_sm")

# Tokenizálás és lemmatizálás
doc = nlp(text)
lemmatized_spacy = [token.lemma_ for token in doc][:20]

print("\nSpaCy lemmatizált szavak:", lemmatized_spacy)


Mit csinálunk?

- Tokenizáljuk a szöveget és automatikusan lemmatizáljuk.
- SpaCy pontosabb, mert a szavak szófaját is figyelembe veszi.

### Stemming és Lemmatizálás vizualizálása

Vizsgáljuk meg a különbséget egy oszlopdiagram segítségével!

In [ ]:
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from nltk.stem import PorterStemmer

# SpaCy NLP modell betöltése
nlp = spacy.load("en_core_web_sm")

# Stemming inicializálása NLTK-val
porter = PorterStemmer()

# Teszt szavak listája
words = ["running", "better", "flies", "talking", "ate", "mice", "swimming", "driving", "playing", "children"]

# Stemming és Lemmatizálás alkalmazása
stemmed = [porter.stem(word) for word in words]  # NLTK PorterStemmer
lemmatized = [nlp(word)[0].lemma_ for word in words]  # SpaCy lemmatizáció

# Adatok DataFrame-be rendezése
df_viz = pd.DataFrame({"Word": words, "Stemming": stemmed, "Lemmatization": lemmatized})

# Vizualizáció
plt.figure(figsize=(10, 5))

# Stemming (kék oszlopok)
sns.barplot(x=df_viz["Word"], y=[len(word) for word in df_viz["Stemming"]], color="blue", label="Stemming")

# Lemmatization (piros oszlopok)
sns.barplot(x=df_viz["Word"], y=[len(word) for word in df_viz["Lemmatization"]], color="red", label="Lemmatization", alpha=0.7)

# Diagram beállítások
plt.xlabel("Szavak")  # X tengely címke
plt.ylabel("Szó hosszúsága")  # Y tengely címke
plt.legend()  # Jelmagyarázat megjelenítése
plt.title("Stemming vs. Lemmatizálás összehasonlítása")  # Diagram címe

# Diagram megjelenítése
plt.show()



Mit látunk?

- A stemming rövidebb szavakat eredményez, de néha pontatlan.
- A lemmatizálás hosszabb, de pontosabb, mert figyelembe veszi a szófajt is.

### N-gramok


Mi az N-gram és miért fontos?
- Az N-gram egy szavakból álló egymást követő egység a szövegben.
- Példa:
  - Unigram (1-gram): ["I", "love", "machine", "learning"]
  - Bigram (2-gram): [("I", "love"), ("love", "machine"), ("machine", "learning")]
  - Trigram (3-gram): [("I", "love", "machine"), ("love", "machine", "learning")]

Nagyobb N-gramok jobban megőrzik a szöveg jelentését, de több számítási erőforrást igényelnek.



### N-gramok generálása NLTK segítségével
- Készítsünk Unigramokat, Bigramokat és Trigramokat egy szövegen belül.

### SMS Spam Dataset betöltése és előnézet
- Először töltsük be az adatokat és nézzük meg az első pár sort.

In [ ]:
import pandas as pd

url = "https://github.com/mohitgupta-1O1/Kaggle-SMS-Spam-Collection-Dataset-/raw/refs/heads/master/spam.csv"
df = pd.read_csv(url, sep=",", header=0,  encoding='latin1', on_bad_lines='skip').iloc[:,:2].rename(columns={'v1':'label','v2':'message'})

print(df.head())

Mit csinálunk?

- Letöltjük az SMS Spam Datasetet
- A label oszlop tartalmazza az üzenet típusát (ham = normál, spam = reklám)
- A message oszlop az SMS-ek tartalmát

In [ ]:
print(df["message"].value_counts())  # Osztályok eloszlása
print("Adathalmaz mérete:", df.shape)  # Sorok és oszlopok száma


### N-gramok generálása
- Az N-gramokat az SMS-ekre alkalmazzuk (unigram, bigram, trigram).

In [ ]:
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize

# Első SMS üzenet kiválasztása és tokenizálása
sample_text = df['message'][5570]
words = word_tokenize(sample_text)

# N-gramok generálása
unigrams = list(ngrams(words, 1))
bigrams = list(ngrams(words, 2))
trigrams = list(ngrams(words, 3))

print("\nUnigramok:", unigrams[:10])
print("\nBigramok:", bigrams[:10])
print("\nTrigramok:", trigrams[:10])


Mit csinálunk?

- Tokenizáljuk az 5570. SMS-t.
- Generáljuk az N-gramokat (unigram, bigram, trigram).
- Megnézzük az első 10 darabot mindegyikből.

Mikor hasznosak az N-gramok?
-  Bigramok és trigramok megőrzik a szókapcsolatokat
-  Jobb modelleket készíthetünk belőlük, pl. spam detektálásra

### WordCloud – Mi ez és miért hasznos?
- A WordCloud (szófelhő) egy vizualizációs technika, amely a leggyakoribb szavakat jeleníti meg egy szöveges korpuszból.
- A gyakoribb szavak nagyobb méretben jelennek meg, míg a ritkább szavak kisebbek.


Alap WordCloud generálása
- Készítsünk egy egyszerű WordCloud-ot egy szöveges adatból!

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Minta szöveg
text = "WordCloud is a great way to visualize text data and find the most frequent words in a dataset."

# WordCloud objektum létrehozása
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Kirajzolás
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")  # Kikapcsoljuk a tengelyeket
plt.title("Egyszerű WordCloud példa")
plt.show()


Mit csinálunk?

- Generálunk egy szófelhőt egy minta szövegből.
- Fehér háttéren jelenítjük meg a szavakat, a gyakoribb szavak nagyobb méretben látszanak.
- Eredmény: Egy szófelhő, amelyben a "WordCloud" és "visualize" nagyobb, mert ezek a szavak gyakran előfordulnak.

WordCloud a Twitter véleményekhez
- Készítsünk WordCloud-ot a SMS adataira!

In [ ]:
df_spam = " ".join(df[df['label'] == 'spam']["message"])
df_notspam = " ".join(df[df['label'] == 'ham']["message"])

In [ ]:
# WordCloud generálása
plt.figure(figsize=(12, 6))

# Spam sms
plt.subplot(1, 2, 1)
plt.imshow(WordCloud(width=800, height=400, background_color='white').generate(df_spam))
plt.title("Spam - WordCloud")
plt.axis("off")

# Nem spam
plt.subplot(1, 2, 2)
plt.imshow(WordCloud(width=800, height=400, background_color='black', colormap="Reds").generate(df_notspam))
plt.title("Nem Spam - WordCloud")
plt.axis("off")

plt.show()

### Véleménydetekció (Sentiment Analysis) az SMS Spam Dataset-en
- Pozitív/negatív érzelmek felismerése szövegben
- Cél: Az SMS-ek érzelmi tónusának meghatározása NLP modellekkel




Miért fontos a véleménydetekció?

- Segít megérteni a felhasználói visszajelzéseket (pl. ügyfélszolgálati elemzések)
- Használható spam detektálásra (negatív hangvételű üzenetek kiszűrése)
- Automatikusan feldolgozza a szövegeket gépi tanulás segítségével
- Most megmutatjuk, hogyan végezhetünk véleményelemzést három különböző módszerrel.

### Szabályalapú véleménydetekció – VADER (NLTK)
-  A VADER (Valence Aware Dictionary and sEntiment Reasoner) egy szabályalapú eszköz, amely főleg rövid szövegekre, pl. tweetek és SMS-ek elemzésére alkalmas.

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

# VADER inicializálása
sia = SentimentIntensityAnalyzer()

# Vélemény pontszámok kiszámítása
df["sentiment_score"] = df["message"].apply(lambda x: sia.polarity_scores(x)["compound"])

# Pozitív/negatív/semleges besorolás
df["sentiment"] = df["sentiment_score"].apply(lambda x: "positive" if x > 0.05 else ("negative" if x < -0.05 else "neutral"))

# Eredmények megtekintése
print(df[["message", "sentiment_score", "sentiment"]].head())


Mit csinálunk?

- A VADER kiszámolja az érzelmi pontszámot az SMS-ekhez.
- Megkapjuk a kategóriát: "positive", "negative", vagy "neutral".

Ez egy gyors és hatékony módszer rövid szövegekre!

In [ ]:
print(df.loc[10,:]['message'])
print('------')
print(df.loc[10,:]['sentiment'])

In [ ]:
print(df.loc[5551,:]['message'])
print('------')
print(df.loc[5551,:]['sentiment'])


### Mélytanulás alapú véleménydetekció – BERT
A Hugging Face transformers könyvtárral finomhangolt mélytanulási modelleket használhatunk.


In [ ]:
from transformers import pipeline

# BERT alapú sentiment analysis modell betöltése
sentiment_model = pipeline("sentiment-analysis")

In [ ]:
# Véleménydetekció egy SMS-re
sample_text = df["message"].iloc[0]
result = sentiment_model(sample_text)

print(f"\nSMS: {sample_text}")
print(f"Predikció: {result}")

Mit csinálunk?

- Előre betanított BERT modellt használunk érzelem felismerésére.
- Nagyon pontos mélytanulási modell, amely a legjobb eredményeket adja.
- A BERT modellek erőforrás-igényesek, de pontosabbak a szabályalapú és gépi tanulásos modelleknél!-

## Feladatok

### 1. Adatok betöltése és előfeldolgozás

- Töltsd be a Twitter vélemények adathalmazt Pandas segítségével.
- Ellenőrizd, hogy milyen oszlopok vannak benne és hány rekordot tartalmaz.
- Nézd meg az első 5 sort.

link: https://raw.githubusercontent.com/sharmaroshan/Twitter-Sentiment-Analysis/refs/heads/master/train_tweet.csv

### 2. Szövegtisztítás és tokenizálás
- Tisztítsd meg a Twitter szövegeket!
- Távolítsd el a speciális karaktereket, URL-eket és számokat.
- Tokenizáld a mondatokat szavakra.

### 3. Szövegek hossza, histogram
- Határozd meg hogy melyik szöveg milyen hosszú.
- Számold ki, hogy mennyi az átlagos hossza a pozitív és a negatív üzeneteknek.
- Készíts egy-egy histogramot a szövegek hosszának gyakoriságáról a pozitív és a negatív tweetek esetén.

### 4. Szavak vizualizálása (WordCloud)
- Készíts egy WordCloud-ot a leggyakoribb pozitív és negatív szavakról.

### 5. BERT használata véleménydetekcióra
- Használj Hugging Face BERT modellt a Twitter vélemények osztályozására.
- Próbálj ki egy pozitív és egy negatív szöveget.